## Setup: Install Dependencies

In [ ]:
# Installation pour Colab (Python 3.12+)
!pip install --upgrade pip setuptools wheel -q
!pip install transformers[torch] datasets wandb accelerate -q

# Vérification
import torch
import transformers
print("✅ Installation réussie!")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 0. Mount Google Drive

In [ ]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_BASE_PATH = '/content/drive/MyDrive/dpo_ppo_training'
    os.makedirs(SAVE_BASE_PATH, exist_ok=True)
    print(f"✅ Google Drive monté. Modèles sauvegardés sur: {SAVE_BASE_PATH}")
    USE_DRIVE = True
except ImportError:
    SAVE_BASE_PATH = './results'
    USE_DRIVE = False
    print(f"⚠️  Pas de Google Drive détecté. Stockage local: {SAVE_BASE_PATH}")

## 1. Configuration

In [ ]:
import torch
import numpy as np
import wandb
import json
from datetime import datetime
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# =====================================
# CONFIGURATION
# =====================================

# Model choice
BASE_MODEL = "distilbert-base-uncased"  # Léger et rapide (ou "roberta-base")

# Training hyperparameters
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
MAX_LENGTH = 512

# Paths
REWARD_MODEL_PATH = f"{SAVE_BASE_PATH}/reward_model"

print(f"{'='*80}")
print(f"Reward Model Training Configuration")
print(f"{'='*80}")
print(f"Base Model: {BASE_MODEL}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Gradient Accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Max Length: {MAX_LENGTH}")
print(f"Output: {REWARD_MODEL_PATH}")
print(f"{'='*80}\n")

## 2. Load and Prepare Dataset

**Stratégie** :
- Chaque paire (prompt, chosen, rejected) → 2 samples
- Sample 1: (prompt + chosen, label=1)
- Sample 2: (prompt + rejected, label=0)
- Le modèle apprend : chosen > rejected

In [ ]:
print("="*80)
print("ÉTAPE 1: Chargement des paires de préférences")
print("="*80)

# Load preference pairs
pairs_path = f"{SAVE_BASE_PATH}/datasets/preference_pairs.json"

if not os.path.exists(pairs_path):
    raise FileNotFoundError(
        f"❌ Dataset introuvable: {pairs_path}\n"
        "Veuillez d'abord générer les paires de préférences."
    )

print(f"\n📥 Chargement depuis: {pairs_path}")
with open(pairs_path, 'r', encoding='utf-8') as f:
    preference_pairs = json.load(f)

print(f"✅ {len(preference_pairs)} paires chargées")

# Create binary classification dataset
texts = []
labels = []

for pair in preference_pairs:
    # Chosen example (label=1, positive class)
    texts.append(pair["chosen"])
    labels.append(1)
    
    # Rejected example (label=0, negative class)
    texts.append(pair["rejected"])
    labels.append(0)

print(f"\n📊 Dataset créé:")
print(f"   - Total samples: {len(texts)}")
print(f"   - Chosen (label=1): {sum(labels)}")
print(f"   - Rejected (label=0): {len(labels) - sum(labels)}")

# Create HuggingFace dataset
reward_dataset = Dataset.from_dict({
    "text": texts,
    "label": labels
})

# Split train/validation
split_dataset = reward_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print(f"\n✅ Train: {len(train_dataset)}, Validation: {len(val_dataset)}")
print(f"\nExemple de sample:")
print(f"  Text: {train_dataset[0]['text'][:100]}...")
print(f"  Label: {train_dataset[0]['label']}")

## 3. Load Model and Tokenizer

In [ ]:
print("="*80)
print("ÉTAPE 2: Chargement du modèle")
print("="*80)

# Load tokenizer
print(f"\n📥 Chargement du tokenizer: {BASE_MODEL}")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer chargé")

# Load model for binary classification (num_labels=2: chosen vs rejected)
print(f"\n📥 Chargement du modèle: {BASE_MODEL}")
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=2,  # Binary classification: chosen (1) vs rejected (0)
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
)

if torch.cuda.is_available():
    model = model.to("cuda")
    print(f"✅ Modèle chargé sur GPU ({model.num_parameters() / 1e6:.1f}M params)")
else:
    print(f"⚠️  Modèle chargé sur CPU")

print(f"\n{'='*80}")
print(f"✅ Modèle prêt pour l'entraînement")
print(f"{'='*80}\n")

## 4. Tokenize Dataset

In [ ]:
print("Tokenizing dataset...")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False  # Dynamic padding dans DataCollator
    )

# Tokenize
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

print(f"✅ Tokenization terminée")
print(f"   Train samples: {len(tokenized_train)}")
print(f"   Val samples: {len(tokenized_val)}")

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 5. Initialize W&B

In [ ]:
# Login to W&B
wandb.login()

# Initialize W&B run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"reward_model_{timestamp}"

wandb.init(
    project="reward_model",
    name=run_name,
    config={
        "base_model": BASE_MODEL,
        "num_pairs": len(preference_pairs),
        "num_samples": len(texts),
        "batch_size": BATCH_SIZE,
        "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
        "learning_rate": LEARNING_RATE,
        "num_epochs": NUM_EPOCHS,
        "max_length": MAX_LENGTH,
    }
)

print(f"✅ W&B initialized: {run_name}")

## 6. Configure Trainer

In [ ]:
print("="*80)
print("ÉTAPE 3: Configuration du Trainer")
print("="*80)

training_args = TrainingArguments(
    output_dir=REWARD_MODEL_PATH,
    
    # Training
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=100,
    
    # Logging
    logging_steps=10,
    logging_first_step=True,
    report_to=["wandb"],
    
    # Saving
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    
    # Optimization
    fp16=False,
    bf16=True if torch.cuda.is_available() else False,
    
    # Other
    seed=42,
    remove_unused_columns=False,
)

print(f"\n📊 Configuration:")
print(f"   - Epochs: {NUM_EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"   - Effective batch: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"   - Learning rate: {LEARNING_RATE}")
print(f"   - Total steps: ~{len(tokenized_train) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS) * NUM_EPOCHS}")

## 7. Define Metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    """Compute accuracy, precision, recall, F1 for binary classification."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

print("✅ Metrics function defined")

## 8. Train Reward Model

In [ ]:
print("="*80)
print("ÉTAPE 4: Entraînement du Reward Model")
print("="*80)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"\n🚀 Démarrage de l'entraînement...\n")

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Train
trainer.train()

print(f"\n{'='*80}")
print(f"✅ Entraînement terminé!")
print(f"{'='*80}")

## 9. Evaluate Reward Model

In [ ]:
print("="*80)
print("ÉTAPE 5: Évaluation finale")
print("="*80)

# Evaluate
eval_results = trainer.evaluate()

print(f"\n📊 Résultats sur validation:")
for metric, value in eval_results.items():
    print(f"   {metric}: {value:.4f}")

print(f"\n{'='*80}")
print(f"✅ Évaluation terminée")
print(f"{'='*80}")

## 10. Save Reward Model

In [ ]:
print("="*80)
print("ÉTAPE 6: Sauvegarde du modèle")
print("="*80)

# Save model and tokenizer
print(f"\n💾 Sauvegarde dans: {REWARD_MODEL_PATH}")
trainer.save_model(REWARD_MODEL_PATH)
tokenizer.save_pretrained(REWARD_MODEL_PATH)

print(f"\n✅ Reward model sauvegardé!")
print(f"\n📁 Fichiers créés:")
print(f"   - {REWARD_MODEL_PATH}/pytorch_model.bin")
print(f"   - {REWARD_MODEL_PATH}/config.json")
print(f"   - {REWARD_MODEL_PATH}/tokenizer.json")

# Close W&B
wandb.finish()
print(f"\n✅ W&B run closed")

print(f"\n{'='*80}")
print(f"✅ REWARD MODEL TRAINING COMPLETE!")
print(f"{'='*80}")
print(f"\n🎯 Prochaine étape:")
print(f"   Utilisez ce modèle pour PPO avec USE_REWARD_MODEL=True")

## 11. Test Reward Model

In [ ]:
print("="*80)
print("Test du Reward Model")
print("="*80)

# Load model for testing
test_model = AutoModelForSequenceClassification.from_pretrained(REWARD_MODEL_PATH)
test_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_PATH)

if torch.cuda.is_available():
    test_model = test_model.to("cuda")

test_model.eval()

# Test texts
test_texts = [
    "This movie is absolutely amazing and wonderful! I loved every minute of it.",
    "This movie is terrible, boring and a complete waste of time. Awful!",
    "The film was okay, nothing special but not bad either."
]

print(f"\n📊 Prédictions du reward model:\n")

for i, text in enumerate(test_texts, 1):
    # Tokenize
    inputs = test_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_LENGTH
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        outputs = test_model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        
        # Probability of being "chosen" (class 1)
        reward_score = probs[0, 1].item()
        prediction = "CHOSEN" if reward_score > 0.5 else "REJECTED"
    
    print(f"Text {i}: {text[:60]}...")
    print(f"  → Reward Score: {reward_score:.4f}")
    print(f"  → Prediction: {prediction}")
    print()

print(f"{'='*80}")
print(f"✅ Test terminé!")
print(f"\n💡 Le reward model est prêt à être utilisé dans PPO")